In [1]:
using System;
using System.Collections.Generic;
using System.Linq;

public enum OrderStatus
{
    Created,
    Processing,
    Completed
}

public delegate void OrderStatusChangedHandler(Order order, OrderStatus oldStatus, OrderStatus newStatus);
public delegate void EmployeeAssignedHandler(Order order, Employee employee);
public delegate void NotificationHandler(string message);

public class Company
{
    public string Name { get; private set; }
    private List<Employee> employees = new List<Employee>();
    private List<Order> orders = new List<Order>();
    private int orderCounter = 1;

    public event NotificationHandler OnNotification;

    public Company(string name)
    {
        Name = name;
    }

    public void AddEmployee(Employee employee)
    {
        employees.Add(employee);
        Notify($"Сотрудник {employee.FirstName} {employee.LastName} добавлен в компанию");
    }

    public void AddOrder(Order order)
    {
        order.OrderId = orderCounter++;
        orders.Add(order);
        
        order.OnStatusChanged += HandleOrderStatusChanged;
        order.OnEmployeeAssigned += HandleEmployeeAssigned;
        
        Notify($"Заказ №{order.OrderId} добавлен в систему");
    }

    public void AssignEmployeeToOrder(int orderId, string employeeFirstName, string employeeLastName)
    {
        var order = orders.FirstOrDefault(o => o.OrderId == orderId);
        var employee = employees.FirstOrDefault(e => 
            e.FirstName == employeeFirstName && e.LastName == employeeLastName);

        if (order != null && employee != null)
        {
            order.AssignEmployee(employee);
        }
        else
        {
            Notify("Ошибка назначения: заказ или сотрудник не найден");
        }
    }

    public List<Order> GetOrdersByStatus(OrderStatus status)
    {
        return orders.Where(o => o.Status == status).ToList();
    }

    public List<Order> FindOrdersByCustomer(string customerFirstName, string customerLastName)
    {
        return orders.Where(o => 
            o.Customer.FirstName == customerFirstName && 
            o.Customer.LastName == customerLastName).ToList();
    }

    public void DisplayAllOrders()
    {
        Notify($"\nВсе заказы компании {Name}:");
        foreach (var order in orders)
        {
            order.DisplayInfo();
        }
    }

    public void GenerateEmployeeReport()
    {
        Notify($"\nОтчет по сотрудникам компании {Name}:");
        foreach (var employee in employees)
        {
            int assignedOrders = orders.Count(o => o.AssignedEmployee == employee);
            Console.WriteLine($"{employee.FirstName} {employee.LastName} - {employee.Position}");
            Console.WriteLine($"Обработано заказов: {employee.OrdersProcessed}");
            Console.WriteLine($"Назначено заказов: {assignedOrders}");
        }
    }

    private void HandleOrderStatusChanged(Order order, OrderStatus oldStatus, OrderStatus newStatus)
    {
        Notify($"Заказ №{order.OrderId} изменил статус: {oldStatus} - {newStatus}");
    }

    private void HandleEmployeeAssigned(Order order, Employee employee)
    {
        Notify($"Сотрудник {employee.FirstName} {employee.LastName} назначен на заказ №{order.OrderId}");
    }

    private void Notify(string message)
    {
        OnNotification?.Invoke(message);
    }
}

public class Employee
{
    public string FirstName { get; private set; }
    public string LastName { get; private set; }
    public string Position { get; private set; }
    public int OrdersProcessed { get; private set; }

    public event NotificationHandler OnOrderProcessed;

    public Employee(string firstName, string lastName, string position)
    {
        FirstName = firstName;
        LastName = lastName;
        Position = position;
        OrdersProcessed = 0;
    }

    public void ProcessOrder()
    {
        OrdersProcessed++;
        OnOrderProcessed?.Invoke($"Сотрудник {FirstName} {LastName} обработал заказ. Всего: {OrdersProcessed}");
    }

    public void DisplayInfo()
    {
        Console.WriteLine($"{FirstName} {LastName} - {Position} (Обработано: {OrdersProcessed})");
    }
}

public class Customer
{
    public string FirstName { get; private set; }
    public string LastName { get; private set; }
    public string PhoneNumber { get; private set; }

    public Customer(string firstName, string lastName, string phoneNumber)
    {
        FirstName = firstName;
        LastName = lastName;
        PhoneNumber = phoneNumber;
    }

    public void DisplayInfo()
    {
        Console.WriteLine($"{FirstName} {LastName}, тел.: {PhoneNumber}");
    }

    public void UpdatePhoneNumber(string newPhoneNumber)
    {
        PhoneNumber = newPhoneNumber;
        Console.WriteLine($"Номер телефона обновлен: {newPhoneNumber}");
    }
}

public class Order
{
    public int OrderId { get; set; }
    public string Description { get; private set; }
    public Employee AssignedEmployee { get; private set; }
    public Customer Customer { get; private set; }
    public OrderStatus Status { get; private set; }
    public DateTime CreatedAt { get; private set; }

    public event OrderStatusChangedHandler OnStatusChanged;
    public event EmployeeAssignedHandler OnEmployeeAssigned;

    public Order(string description, Customer customer)
    {
        Description = description;
        Customer = customer;
        Status = OrderStatus.Created;
        CreatedAt = DateTime.Now;
    }

    public void ChangeStatus(OrderStatus newStatus)
    {
        OrderStatus oldStatus = Status;
        Status = newStatus;
        
        OnStatusChanged?.Invoke(this, oldStatus, newStatus);

        if (newStatus == OrderStatus.Completed && AssignedEmployee != null)
        {
            AssignedEmployee.ProcessOrder();
        }
    }

    public void AssignEmployee(Employee employee)
    {
        AssignedEmployee = employee;
        
        OnEmployeeAssigned?.Invoke(this, employee);
    }

    public void DisplayInfo()
    {
        string employeeInfo = AssignedEmployee != null ? 
            $"{AssignedEmployee.FirstName} {AssignedEmployee.LastName}" : "Не назначен";
        
        Console.WriteLine($"Заказ №{OrderId}: {Description}");
        Console.WriteLine($"Статус: {Status}, Создан: {CreatedAt:dd.MM.yyyy}");
        Console.WriteLine($"Заказчик: {Customer.FirstName} {Customer.LastName}");
        Console.WriteLine($"Ответственный: {employeeInfo}");
    }

    public void UpdateDescription(string newDescription)
    {
        Description = newDescription;
        Console.WriteLine($"Описание заказа #{OrderId} обновлено");
    }
}


Company company = new Company("ТехноЛогика");


Employee emp1 = new Employee("Иван", "Иванов", "Менеджер");
Employee emp2 = new Employee("Надежда", "Надеждина", "Разработчица");
Employee emp3 = new Employee("Алексей", "Алексеев", "Тестировщик");

emp1.OnOrderProcessed += (message) => Console.WriteLine($"[сотрудник] {message}");
emp2.OnOrderProcessed += (message) => Console.WriteLine($"[сотрудник] {message}");
emp3.OnOrderProcessed += (message) => Console.WriteLine($"[сотрудник] {message}");

company.AddEmployee(emp1);
company.AddEmployee(emp2);
company.AddEmployee(emp3);

Customer customer1 = new Customer("Петр", "Петров", "+7-123-456-7890");
Customer customer2 = new Customer("Елена", "Еленина", "+7-987-654-3210");

Order order1 = new Order("Разработка веб-сайта", customer1);
Order order2 = new Order("Мобильное приложение", customer2);
Order order3 = new Order("Техническая поддержка", customer1);

company.AddOrder(order1);
company.AddOrder(order2);
company.AddOrder(order3);

company.AssignEmployeeToOrder(1, "Иван", "Иванов");
company.AssignEmployeeToOrder(2, "Надежда", "Надеждина");
company.AssignEmployeeToOrder(3, "Алексей", "Алексеев");

order1.ChangeStatus(OrderStatus.Processing);
order2.ChangeStatus(OrderStatus.Processing);

order1.ChangeStatus(OrderStatus.Completed);
order3.ChangeStatus(OrderStatus.Completed);

var processingOrders = company.GetOrdersByStatus(OrderStatus.Processing);
foreach (var order in processingOrders)
{
    order.DisplayInfo();
}

Console.WriteLine("\n заказы Елены Елениной");
var customerOrders = company.FindOrdersByCustomer("Елена", "Еленина");
foreach (var order in customerOrders)
{
    order.DisplayInfo();
}

company.GenerateEmployeeReport();

Console.WriteLine("\n все заказы");
company.DisplayAllOrders();

[сотрудник] Сотрудник Иван Иванов обработал заказ. Всего: 1
[сотрудник] Сотрудник Алексей Алексеев обработал заказ. Всего: 1
Заказ №2: Мобильное приложение
Статус: Processing, Создан: 03.12.2025
Заказчик: Елена Еленина
Ответственный: Надежда Надеждина

 заказы Елены Елениной
Заказ №2: Мобильное приложение
Статус: Processing, Создан: 03.12.2025
Заказчик: Елена Еленина
Ответственный: Надежда Надеждина
Иван Иванов - Менеджер
Обработано заказов: 1
Назначено заказов: 1
Надежда Надеждина - Разработчица
Обработано заказов: 0
Назначено заказов: 1
Алексей Алексеев - Тестировщик
Обработано заказов: 1
Назначено заказов: 1

 все заказы
Заказ №1: Разработка веб-сайта
Статус: Completed, Создан: 03.12.2025
Заказчик: Петр Петров
Ответственный: Иван Иванов
Заказ №2: Мобильное приложение
Статус: Processing, Создан: 03.12.2025
Заказчик: Елена Еленина
Ответственный: Надежда Надеждина
Заказ №3: Техническая поддержка
Статус: Completed, Создан: 03.12.2025
Заказчик: Петр Петров
Ответственный: Алексей Алексеев